xgboost


In [12]:

import numpy as np 
import pandas as pd 
import seaborn as sb
import xgboost as xgb

from sklearn import metrics
from sklearn.utils import shuffle

In [13]:
raw_train_data = pd.read_csv("D:\\train.csv")
raw_test_data = pd.read_csv("D:\\test.csv")

In [14]:
print(raw_train_data.shape)
print(raw_test_data.shape)

(61878, 95)
(144368, 94)


In [15]:
Train_data = raw_train_data.iloc[:, 1:95]
labels = set(Train_data.iloc[:,93:94].target)
Valid_x = np.array(raw_test_data.iloc[:, 1:94])
Total_x = np.array(Train_data.iloc[:,0:93])
Total_y = np.array(Train_data.iloc[:,93:94])

In [16]:
print(f"The current has the following label {labels}")
print(f"The shape the Total_X {Total_x.shape}")
print(f"The shape the Total_Y {Total_y.shape}")
print(f"The shape the Valid_X {Valid_x.shape}")
print(f"Check Nan in Total_X {set(np.isnan(Total_x).any(axis=1))}")
# print(f"Check Nan in Train_Y {np.isnan(Train_Y).any(axis=1)}")
print(f"Check Nan in Valid_X {set(np.isnan(Valid_x).any(axis=1))}")

The current has the following label {'Class_1', 'Class_4', 'Class_2', 'Class_8', 'Class_6', 'Class_9', 'Class_3', 'Class_7', 'Class_5'}
The shape the Total_X (61878, 93)
The shape the Total_Y (61878, 1)
The shape the Valid_X (144368, 93)
Check Nan in Total_X {False}
Check Nan in Valid_X {False}


In [17]:
Total_X,Total_Y = shuffle(Total_x,Total_y)

In [18]:
# In this part using the previous 50000 to do the training and the last data to do the test check the score
Train_X = Total_X[:50000]
Train_Y = Total_Y[:50000]
Test_X = Total_X[50000:]
Test_Y = Total_Y[50000:]
print(Train_X.shape)
print(Train_Y.shape)
print(Test_X.shape)
print(Test_Y.shape)

(50000, 93)
(50000, 1)
(11878, 93)
(11878, 1)


In [34]:
Train_Y_num = list()
for item in Train_Y:
    Train_Y_num.append(int(item[-1][-1])-1)
# print(Train_Y_num)

In [35]:
Training_data_xgb = xgb.DMatrix(Train_X,Train_Y_num)
Testing_data_xgb = xgb.DMatrix(Test_X)

In [36]:
params = {"objective": "multi:softprob", "eval_metric":"mlogloss", "num_class": 9}

In [37]:
#number of iterations - initial 20, 
gbm = xgb.train(params, Training_data_xgb, 180)
# predict_Y_xgb = gbm.predict(Testing_data_xgb)

In [38]:
predict_Y_xgb = gbm.predict(Testing_data_xgb)

In [56]:
print(predict_Y_xgb)

[[1.0659158e-03 7.9918426e-01 1.7152654e-01 ... 4.9858661e-03
  1.2911009e-02 6.0700788e-03]
 [4.3151355e-03 1.2497307e-05 2.4305460e-05 ... 8.7789027e-04
  7.0006069e-04 2.0085921e-04]
 [1.6144068e-03 3.8237486e-04 1.8052303e-03 ... 7.1261108e-01
  3.7561811e-03 4.2994102e-04]
 ...
 [1.1210376e-02 3.5634160e-04 9.0564962e-04 ... 1.4968207e-03
  7.1794111e-01 2.0267349e-01]
 [7.6303883e-03 8.3482351e-05 8.8830064e-05 ... 2.1254100e-06
  5.6644556e-05 9.8909783e-01]
 [2.2344955e-06 1.8657406e-03 2.8593387e-05 ... 2.9814576e-06
  5.8801816e-06 1.2215107e-05]]


In [75]:
result_list = list()
for item in predict_Y_xgb:
    result_list.append(np.where(item==max(item))[0].tolist()[0]+1)
# print(result_list)


In [76]:
ground_true_label_Y = [int(item[-1][-1]) for item in Test_Y]
# print(ground_true_label_Y)

In [ ]:
normal_svm_predict = metrics.classification_report(ground_true_label_Y,result_list)
print(normal_svm_predict)